<a href="https://colab.research.google.com/github/Aadhimozhi/Intelligence-System-/blob/main/Intelligence_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q google-generativeai numpy

In [22]:
import numpy as np
import re
from sentence_transformers import SentenceTransformer

# ==============================
# CONFIG
# ==============================

MODEL_NAME = "all-MiniLM-L6-v2"   # Local AI embedding model
TOP_K = 3

model = SentenceTransformer(MODEL_NAME)

# ==============================
# LOAD NOTES
# ==============================

def load_notes(path="notes.txt"):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

# ==============================
# TEXT CHUNKING
# ==============================

def chunk_text(text, size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

# ==============================
# EMBEDDINGS
# ==============================

def embed_texts(texts):
    return model.encode(texts, convert_to_numpy=True)

# ==============================
# COSINE SIMILARITY
# ==============================

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# ==============================
# SEMANTIC SEARCH
# ==============================

def semantic_search(query, chunks, embeddings, k=3):
    query_emb = model.encode(query, convert_to_numpy=True)
    scores = [cosine_similarity(query_emb, emb) for emb in embeddings]
    top_idx = np.argsort(scores)[-k:][::-1]
    return [chunks[i] for i in top_idx]

# ==============================
# SHORT ANSWER GENERATION
# ==============================

def generate_answer(question, retrieved_chunks):
    if not retrieved_chunks:
        return "Information not available in the notes."

    # Split chunks into sentences
    sentences = []
    for chunk in retrieved_chunks:
        sentences.extend(re.split(r'(?<=[.!?])\s+', chunk))

    # Extract keywords
    keywords = question.lower().split()

    # Score sentences
    scored = []
    for sent in sentences:
        score = sum(1 for word in keywords if word in sent.lower())
        if score > 0:
            scored.append((score, sent.strip()))

    if not scored:
        return "Information not available in the notes."

    # Sort by relevance
    scored.sort(reverse=True)

    # Return top 2 sentences only
    answer = " ".join([s[1] for s in scored[:2]])
    return answer

# ==============================
# QA PIPELINE
# ==============================

def answer_question(question, chunks, embeddings):
    retrieved = semantic_search(question, chunks, embeddings, TOP_K)

    if not retrieved:
        return "Information not available in the notes."

    answer = generate_answer(question, retrieved)

    if "Information not available" in answer:
        return answer

    return answer + "\n\nSource: " + ", ".join(
        [f"Chunk {i+1}" for i in range(len(retrieved))]
    )

# ==============================
# RUN SYSTEM
# ==============================

notes = load_notes()
chunks = chunk_text(notes)

print("📐 Creating AI embeddings...")
embeddings = embed_texts(chunks)

print("✅ System Ready (type 'exit' to stop)\n")

while True:
    q = input("❓ Question: ")
    if q.lower() == "exit":
        print("👋 System stopped.")
        break

    print("\n🧠 Answer:")
    print(answer_question(q, chunks, embeddings))
    print("-" * 60)


📐 Creating AI embeddings...
✅ System Ready (type 'exit' to stop)

❓ Question: Who invented transformers?

🧠 Answer:
Information not available in the notes.
------------------------------------------------------------
❓ Question: exit
👋 System stopped.
